In [9]:
# Import Libraries
import numpy as np
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split, Subset
from torchvision import models, transforms
from sklearn.preprocessing import StandardScaler

# Check GPU Availability
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load and Prepare Data
train_df = pd.read_csv('./content/train.csv')
test_df = pd.read_csv('./content/test.csv')

# Define Dataset and DataLoader
class PlantDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, is_test=False, target_trait=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        self.target_trait = target_trait
        self.label_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']
        
        # Separate features and labels
        if self.is_test:
            self.features = self.data.drop(columns=['id'])
            self.labels = pd.DataFrame(columns=self.label_columns)
        else:
            self.features = self.data.drop(columns=['id'] + self.label_columns)
            self.labels = self.data[self.label_columns]
        
        # Initialize StandardScaler
        self.scaler = StandardScaler()
        self.features = pd.DataFrame(self.scaler.fit_transform(self.features), columns=self.features.columns)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, f"{self.data.iloc[idx, 0]}.jpeg")
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        features = self.features.iloc[idx].values.astype(np.float32)

        if self.is_test:
            label = np.array([0], dtype=np.float32)  # Dummy label for test data
        else:
            if self.target_trait:
                label = np.array([self.labels.iloc[idx][self.target_trait]], dtype=np.float32)
            else:
                label = self.labels.iloc[idx].values.astype(np.float32)

        return image, features, label

# Image transformations
transform = transforms.Compose([
        transforms.Resize(224, interpolation=3),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

# Define Model Architecture
class PlantTraitPredictor(nn.Module):
    def __init__(self):
        super(PlantTraitPredictor, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 256)
        self.fc1 = nn.Linear(256 + 163, 128)
        self.fc2 = nn.Linear(128, 1)  # Predict a single trait

    def forward(self, image, ancillary_data):
        img_features = self.resnet(image)
        combined = torch.cat((img_features, ancillary_data), dim=1)
        x = nn.ReLU()(self.fc1(combined))
        x = self.fc2(x)
        return x



True
NVIDIA GeForce RTX 4090
Using device: cuda:0


In [11]:
# Training Function
def train_model(trait, num_epochs=25, batch_size=32, learning_rate=0.001):
    # Load the CSV file to get access to all the data
    df = pd.read_csv('./content/train.csv')
    
    # Calculate the percentiles for the target trait
    trait_data = df[trait]
    lower_bound = np.percentile(trait_data, 0.1)
    upper_bound = np.percentile(trait_data, 98)
    
    # Create a mask for the dataset
    mask = (trait_data > lower_bound) & (trait_data < upper_bound)
    
    # Apply the mask to get indices of non-outlier data
    valid_indices = np.where(mask)[0]

    # Create the full dataset
    full_dataset = PlantDataset(csv_file='./content/train.csv', img_dir='./content/train_images', transform=transform, target_trait=trait)

    # Create a new dataset with only the non-outlier data
    filtered_dataset = Subset(full_dataset, valid_indices)
    
    # Split into training and validation sets
    train_size = int(0.8 * len(filtered_dataset))
    val_size = len(filtered_dataset) - train_size
    train_dataset, val_dataset = random_split(filtered_dataset, [train_size, val_size])

    # smaller datasrt4
    #train_dataset = Subset(train_dataset, range(3200))
    #val_dataset = Subset(val_dataset, range(3200))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Print statistics about the filtering
    print(f"Original dataset size: {len(full_dataset)}")
    print(f"Filtered dataset size: {len(filtered_dataset)}")
    print(f"Removed {len(full_dataset) - len(filtered_dataset)} samples ({(1 - len(filtered_dataset)/len(full_dataset))*100:.2f}%)")

    model = PlantTraitPredictor().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

    best_val_loss = float('inf')
    patience = 7
    counter = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, ancillary_data, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, ancillary_data, labels = images.to(device), ancillary_data.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images, ancillary_data)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, ancillary_data, labels in val_loader:
                images, ancillary_data, labels = images.to(device), ancillary_data.to(device), labels.to(device)
                outputs = model(images, ancillary_data)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), f'./content/best_model_{trait}.pth')
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

    return model


# Train Models for Each Trait
traits = ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']

trained_models = {}

# for trait, index in traits:
#     print(f"\nTraining model for {trait}")
#     model = train_model(trait, num_epochs=epochs[index], batch_size=32)
#     trained_models[trait] = model

index = 0

def train_one(index, epochs=30):
    trait = traits[index]
    print(f"\nTraining model for {trait}")
    model = train_model(trait, num_epochs=epochs, batch_size=32)
    trained_models[trait] = model


In [12]:
train_one(0)


Training model for X4_mean
Original dataset size: 43363
Filtered dataset size: 42448
Removed 915 samples (2.11%)


Epoch 1/30: 100%|██████████| 1062/1062 [01:39<00:00, 10.69it/s]


Epoch [1/30], Train Loss: 0.0220, Val Loss: 0.0241


Epoch 2/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.29it/s]


Epoch [2/30], Train Loss: 0.0155, Val Loss: 0.0159


...

Epoch [28/30], Train Loss: 0.0136, Val Loss: 0.0138


Epoch 29/30: 100%|██████████| 1062/1062 [01:32<00:00, 11.45it/s]


Epoch [29/30], Train Loss: 0.0136, Val Loss: 0.0138


Epoch 30/30: 100%|██████████| 1062/1062 [01:31<00:00, 11.58it/s]


Epoch [30/30], Train Loss: 0.0136, Val Loss: 0.0138


In [13]:
train_one(1)


Training model for X11_mean
Original dataset size: 43363
Filtered dataset size: 42450
Removed 913 samples (2.11%)


Epoch 1/30: 100%|██████████| 1062/1062 [01:36<00:00, 11.06it/s]


Epoch [1/30], Train Loss: 319.6569, Val Loss: 64.0803


Epoch 2/30: 100%|██████████| 1062/1062 [01:35<00:00, 11.08it/s]


Epoch [2/30], Train Loss: 49.0624, Val Loss: 80.7666


...

Epoch [28/30], Train Loss: 31.6012, Val Loss: 32.2218


Epoch 29/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.23it/s]


Epoch [29/30], Train Loss: 31.6320, Val Loss: 32.2158


Epoch 30/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.24it/s]


Epoch [30/30], Train Loss: 31.5514, Val Loss: 32.7479


In [14]:
train_one(2, 30)


Training model for X18_mean
Original dataset size: 43363
Filtered dataset size: 42442
Removed 921 samples (2.12%)


Epoch 1/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.27it/s]


Epoch [1/30], Train Loss: 16704864.5887, Val Loss: 569580.7493


Epoch 2/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.28it/s]


Epoch [2/30], Train Loss: 20619.0316, Val Loss: 3783318.1212


...

Epoch [15/30], Train Loss: 14.7557, Val Loss: 660.4821


Epoch 16/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.23it/s]


Epoch [16/30], Train Loss: 13.4155, Val Loss: 8470.0273


Epoch 17/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.25it/s]


Epoch [17/30], Train Loss: 13.9413, Val Loss: 2308.2051
Early stopping triggered at epoch 17


In [15]:
train_one(3)


Training model for X26_mean
Original dataset size: 43363
Filtered dataset size: 42447
Removed 916 samples (2.11%)


Epoch 1/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.23it/s]


Epoch [1/30], Train Loss: 295911.7853, Val Loss: 124692.0570


Epoch 2/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.27it/s]


Epoch [2/30], Train Loss: 3557.3705, Val Loss: 1770667.3543


...

Epoch [28/30], Train Loss: 1092.7055, Val Loss: 1450.7161


Epoch 29/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.24it/s]


Epoch [29/30], Train Loss: 1102.3087, Val Loss: 2189.9543


Epoch 30/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.27it/s]


Epoch [30/30], Train Loss: 1090.2494, Val Loss: 1739.4881


In [16]:
train_one(4)


Training model for X50_mean
Original dataset size: 43363
Filtered dataset size: 42450
Removed 913 samples (2.11%)


Epoch 1/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.24it/s]


Epoch [1/30], Train Loss: 2.0695, Val Loss: 0.6831


Epoch 2/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.26it/s]


Epoch [2/30], Train Loss: 0.4070, Val Loss: 0.6664


...

Epoch [28/30], Train Loss: 0.2579, Val Loss: 0.2637


Epoch 29/30: 100%|██████████| 1062/1062 [01:33<00:00, 11.32it/s]


Epoch [29/30], Train Loss: 0.2575, Val Loss: 0.2673


Epoch 30/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.29it/s]


Epoch [30/30], Train Loss: 0.2574, Val Loss: 0.2654


In [17]:
train_one(5)


Training model for X3112_mean
Original dataset size: 43363
Filtered dataset size: 42443
Removed 920 samples (2.12%)


Epoch 1/30: 100%|██████████| 1062/1062 [01:35<00:00, 11.16it/s]


Epoch [1/30], Train Loss: 26400946910.3579, Val Loss: 155054777.1429


Epoch 2/30: 100%|██████████| 1062/1062 [01:35<00:00, 11.13it/s]


Epoch [2/30], Train Loss: 5313978.9495, Val Loss: 2123945747.2481


...

Epoch [28/30], Train Loss: 2484338.5453, Val Loss: 3823958.6128


Epoch 29/30: 100%|██████████| 1062/1062 [01:35<00:00, 11.13it/s]


Epoch [29/30], Train Loss: 2482299.4992, Val Loss: 2566305.7784


Epoch 30/30: 100%|██████████| 1062/1062 [01:34<00:00, 11.23it/s]


Epoch [30/30], Train Loss: 2479777.0363, Val Loss: 2603236.3610


In [18]:
def predict(model, test_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, ancillary_data, _ in tqdm(test_loader):
            images, ancillary_data = images.to(device), ancillary_data.to(device)
            outputs = model(images, ancillary_data)
            predictions.extend(outputs.cpu().numpy())
    return predictions

# Load and preprocess test data
test_df = pd.read_csv('./content/test.csv')
label_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']
features = test_df.drop(columns=['id'])

# Use the scaler fitted on training data
scaler = StandardScaler()
scaler.fit(pd.read_csv('./content/train.csv').drop(columns=['id'] + label_columns))
scaled_features = scaler.transform(features)

# Create test dataset and dataloader
test_dataset = PlantDataset(csv_file='./content/test.csv', img_dir='./content/test_images', transform=transform, is_test=True)
test_dataset.features = pd.DataFrame(scaled_features, columns=features.columns)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Make predictions for each trait
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
predictions = {}
traits = ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']
# traits = ['X11_mean']

for trait in traits:
    model = PlantTraitPredictor().to(device)
    model.load_state_dict(torch.load(f'./content/best_model_{trait}.pth'))
    predictions[trait] = predict(model, test_loader, device)

# Create submission file
submission_df = pd.DataFrame({'id': test_df['id']})
for trait in traits:
    submission_df[trait.replace('_mean', '')] = predictions[trait]

submission_df.to_csv('./content/submission.csv', index=False)
print("Submission file created: ./content/submission.csv")

print(submission_df.describe())

100%|██████████| 200/200 [00:13<00:00, 15.28it/s]


Submission file created: ./content/submission.csv
                 id
count  6.391000e+03
mean   1.706187e+08
std    4.190301e+07
min    1.215100e+05
25%    1.648002e+08
50%    1.913817e+08
75%    1.950978e+08
max    1.967498e+08
